# `LibreASR`

## Imports

In [ ]:
%reload_ext autoreload
%autoreload 3
# %matplotlib inline

In [ ]:
from libreasr.lib.imports import *

## `Config`

In [ ]:
conf, lang, builder_train, builder_valid, db, m, learn = parse_and_apply_config()
(conf, lang, db, m, learn)

In [ ]:
builder_train.print()
builder_valid.print()

In [ ]:
builder_train.plot()

In [ ]:
tpl = db.one_batch()
X, Ym, _, _ = tpl[0]
Y, Y_lens, X_lens = tpl[1]
what(X), what(X_lens), what(Y), what(Y_lens)

In [ ]:
X_lens, Y_lens

### `Augmentation`

In [ ]:
# db.aug(n=5, only=10) #0: signal, 7: spectro

## `Model`

In [ ]:
# switch to gpu mode
# m.convert_to_gpu()

In [ ]:
m = m.to(conf["cuda"]["device"])

In [ ]:
print(m)
print("n_params:", n_params(m))

Timing

In [ ]:
%%time
_in = db.one_batch()[0]

In [ ]:
"""
%%time
m.eval()
with torch.no_grad():
    _out = m(cudaize(_in))
    print("X", _in[0].shape, "->", _out.shape)
    print("Y", _in[1].shape)
"""
1

## `Train`

In [ ]:
ckpt = "best_wer"
ckpt = "libreasr-de-24-12-2020"
ckpt = "model"
ckpt_load = "de"
# ckpt = "model"
# ckpt = "chkp-en-57-wer"

In [ ]:
ns = learn.model
try:
    if conf["training"]["noisystudent"]:
        learn.model = ns.teacher[0]
        learn.load(ckpt_load, with_opt=False, strict=False)
        learn.model = ns
        learn.create_opt()
    else:
        learn.load(ckpt_load, with_opt=False, strict=False) # OPT TRUE
except Exception as e:
    print("failed loading ckpt", ckpt)
    print(e)
finally:
    if conf["training"]["noisystudent"]:
        learn.model = ns
        learn.create_opt()

In [ ]:
%%time
try:
    m.eval()
    with torch.no_grad():
        if conf["training"]["noisystudent"]:
            _out = m.teacher[0].transcribe_batch(cudaize(_in))
        else:
            _out = m.transcribe_batch(cudaize(_in))
    _ = [print(x) for x in _out]
except:
    pass

In [ ]:
# %%time
# m.train()
# _out = m(cudaize(_in))
# _out.shape

### `fit`

In [ ]:
learn.unfreeze()

In [ ]:
# learn.lr_find()

In [ ]:
# hypers
do_warmup = True
epochs = 20
warmup_epochs = 2.9
warmup = int(warmup_epochs + 1)

lr = 4e-4 # 7e-4
wd = 0.01 # 0.1

div = 500. # 100.
div_final = 1.001

pct_start = warmup_epochs / warmup

In [ ]:
if do_warmup:
    print("STAGE: WARMUP")
    try:
        learn.fit_one_cycle(warmup, lr, wd=wd, cbs=learn.extra_cbs, div_final=div_final, div=div, pct_start=pct_start)
    except: pass
    m = m.to(conf["cuda"]["device"])
    learn.save(ckpt, with_opt=True)

In [ ]:
print("STAGE: TUNE")
m = m.to(conf["cuda"]["device"])
# learn.load(ckpt, with_opt=True, strict=False, device=conf["cuda"]["device"])
learn.fit(epochs, lr, wd=wd, cbs=learn.extra_cbs)

In [ ]:
raise Exception("done")

### `checks`

In [ ]:
learn.recorder.values

In [ ]:
# save model for later
# learn.save(checkpoints[-1], with_opt=True)
learn.save(ckpt, with_opt=False)

In [ ]:
learn.save("libreasr-de-24-12-2020", with_opt=False)

## `Decode`

In [ ]:
sampls = 50 # 999999
test_name = "dev-cv"

In [ ]:
_ = list(learn.test(min_samples=sampls, train=False, save_best=False))

In [ ]:
ylens = map(lambda x: x.get("text/ground_truth"), _)
ylens = filter(lambda x: x is not None and len(x) != 0, ylens)
ylens = map(lambda x: len(x), ylens)
ylens = np.array(list(ylens))

wers = map(lambda x: x.get("metrics/wer", None), _)
wers = filter(lambda x: x is not None, wers)
wers = np.array(list(wers))
ylens[:3], wers[:3], len(ylens), len(wers)

In [ ]:
stack = np.stack([ylens, wers]).T
stack.shape

In [ ]:
df = pd.DataFrame(stack, columns=["ylen", "wer"])
df.head()

In [ ]:
mean_wer = df.groupby("ylen").mean()["wer"].values
std_wer = df.groupby("ylen").std()["wer"].values
border_upper = mean_wer + std_wer / 2
border_lower = mean_wer - std_wer / 2
count = df.groupby("ylen").count()["wer"].values

In [ ]:
fig = plt.figure()
plot_wer = fig.add_subplot(111)
plot_count = plot_wer.twinx()
x = np.arange(len(mean_wer))

# wer
plot_wer.plot(mean_wer)
plot_wer.set_ylim(0.0, 1.2)
plot_wer.set_ylabel("WER")
plot_wer.set_xlabel("Label Length")

# std
plot_wer.fill_between(x, border_upper, border_lower, alpha=0.2)

# count
plot_count.bar(x, count, alpha=0.1)
plot_count.set_ylabel("Number of Examples")
plot_count.set_yscale("linear")

# save plot
plt.savefig(f"{test_name}.png", dpi=200)

In [ ]:
# save tests
import pickle
pickle.dump(_, open(f'{test_name}.pkl', 'wb'))

In [ ]:
_[:2]

In [ ]:
def _sort(results, rev):
    results = filter(lambda x: "text/ground_truth" in list(x.keys()), results)
    results = sorted(results, key=lambda x: x["metrics/wer"], reverse=rev)
    return results

def best(results):
    return _sort(results, rev=False)

def worst(results, n=25):
    results = _sort(results, rev=True)[:n]
    df = builder_valid.df
    for res in results:
        def filter_fn(row):
            s1 = (res["text/ground_truth"])
            s2 = sanitize_str(row.label)
            in_ok = s1 in s2
            l_ok = abs(len(s1) - len(s2)) < 10
            return in_ok and l_ok
        rows = df[df.apply(filter_fn, axis=1)]
        row = rows
        print(len(row))
        if len(row) >= 1:
            row = row.iloc[0]
            file = row.file
            sr = int(row.sr)
            xstart = int(row.xstart / 1000. * sr)
            xlen = int(row.xlen / 1000. * sr)
            data, sr = torchaudio.load(file, frame_offset=xstart, num_frames=xlen)
            print("PRED:", res["text/prediction"])
            print("TRUE:", res["text/ground_truth"])
            print(f"WER:  {res['metrics/wer'] * 100.:.2f}%")
            display(Audio(data.numpy(), rate=sr))
        else: print("err")

In [ ]:
worst(_)

In [ ]:
m.joint.joint[2].bias

In [ ]:
m.joint.joint[2].bias

In [ ]:
learn.save("libreasr-es-20-12-2020", with_opt=False)

## Archive

In [ ]:
from lib.model_utils import *
from pathlib import Path

In [ ]:
save_asr_model()